In [17]:
import os
HOME = os.getcwd()
print(HOME)

/home/skyarrow/Workspace/Yolov8-Drillbit-Detection/Predict


In [18]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO

from IPython.display import display, Image

%cd {HOME}/Images/Image_Raw/


# Image(filename='predict.jpg', height = 600)

Ultralytics YOLOv8.0.196 🚀 Python-3.9.18 torch-2.2.0 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 7949MiB)
Setup complete ✅ (20 CPUs, 15.4 GB RAM, 147.1/195.8 GB disk)


/home/skyarrow/Workspace/Yolov8-Drillbit-Detection/Predict/Images/Image_Raw


In [19]:
%cd {HOME}/Images/Image_Crop/
%rm -rf *
%cd {HOME}/Images/Image_Crop_Predicted/
%rm -rf *

/home/skyarrow/Workspace/Yolov8-Drillbit-Detection/Predict/Images/Image_Crop


/home/skyarrow/Workspace/Yolov8-Drillbit-Detection/Predict/Images/Image_Crop_Predicted


# 1. Crop Images

In [20]:
!mkdir -p {HOME}/Images/Image_Crop

import cv2
import os
import sys
import numpy as np
import glob
from multiprocessing import Pool
from functools import partial
import xml.etree.ElementTree as ET
from xml.dom.minidom import Document
from lxml import etree

image_crop = []
image_crop_topleft = []
image_original = []

In [21]:
def Crop_Image(imgname, dirsrc, dirdst, class_dict, subsize=800, gap=200, iou_thresh=0.3, ext='.png'):
    """
    imgname:   待裁切图像名（带扩展名）
    dirsrc:    待裁切的图像保存目录的上一个目录，默认图像与标注文件在一个文件夹下，图像在images下，标注在labelTxt下，标注文件格式为每行一个gt,
               格式为xmin,ymin,xmax,ymax,class,想读其他格式自己动手改
    dirdst:    裁切的图像保存目录的上一个目录，目录下有images,labelTxt两个目录分别保存裁切好的图像或者txt文件，
               保存的图像和txt文件名格式为 oriname_min_ymin.png(.txt),(xmin,ymin)为裁切图像在原图上的左上点坐标,txt格式和原文件格式相同
    subsize:   裁切图像的尺寸，默认为正方形，想裁切矩形自己动手改
    gap:       相邻行或列的图像重叠的宽度
    iou_thresh:小于该阈值的BBGT不会保存在对应图像的txt中（在图像过于边缘或与图像无交集）
    ext:       保存图像的格式
    """
    img = cv2.imread(os.path.join(os.path.join(dirsrc,'Image_Raw'), imgname), -1)
    img_h,img_w = img.shape[:2]
    
    image_original.append(img)
    print(img_h,img_w)
    # Image(img, height = 600)
    
    top = 0
    reachbottom = False
    while not reachbottom:
        reachright = False
        left = 0
        if top + subsize>=img_h:
            reachbottom = True
            top = max(img_h - subsize, 0)
        while not reachright:
            if left + subsize >= img_w:
                reachright = True
                left = max(img_w - subsize, 0)
                
            # imgsplit = img[top:min(top + subsize, img_h), left:min(left + subsize, img_w)].copy()
            imgsplit = img[top:min(top + subsize, img_h), left:min(left + subsize, img_w)]
            
            if imgsplit.shape[:2] != (subsize,subsize):
                template = np.zeros((subsize,subsize,3),dtype=np.uint8)
                template[0:imgsplit.shape[0],0:imgsplit.shape[1]] = imgsplit
                # imgsplit = template.copy()
                imgsplit = template

            cv2.imwrite(os.path.join(dirdst, imgname.split('.')[0] + '_' + str(left) + '_' + str(top) + ext), imgsplit)
            image_crop.append(imgsplit)
            image_crop_topleft.append((top, left))
            left += subsize-gap
        top+=subsize-gap

In [22]:
import tqdm
dirsrc= f'{HOME}/Images'  #'/home/skyarrow/Workspace/Drillbit-Detection/Calculate_Numbers'      #待裁剪图像所在目录的上级目录，图像在JPEGImages文件夹下，标注文件在Annotations下
dirdst= f'{HOME}/Images/Image_Crop'   #裁剪结果存放目录，格式和原图像目录一样

if not os.path.exists(dirdst):
    os.mkdir(dirdst)
if not os.path.exists(dirdst):
    os.mkdir(dirdst)


class_dict = {'0': 0,'1': 1, '2': 2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, '10':10}
name_dict = {'0': 'tool', '1': 'tool_b', '2': 'void'}
subsize = 640  #裁切图像的尺寸 原来是512
gap = 300 #相邻行或列的图像重叠的宽度
iou_thresh = 0.4
ext = '.jpg'
num_thresh = 8

imglist = glob.glob(f'{dirsrc}/Image_Raw/*.jpg')
imgnameList = [os.path.split(imgpath)[-1] for imgpath in imglist]
for imgname in tqdm.tqdm(imgnameList):
    print(imgname)
    Crop_Image(imgname, dirsrc, dirdst, class_dict, subsize, gap, iou_thresh, ext)

  0%|          | 0/1 [00:00<?, ?it/s]

6_3_n.jpg
3692 5544


100%|██████████| 1/1 [00:01<00:00,  1.06s/it]


# 2. Predict & Assemble

In [23]:
def Calculate_IOU(box1, box2):
    """
    box1: (xmin, ymin, xmax, ymax)
    box2: (xmin, ymin, xmax, ymax)
    """
    x1, y1, x2, y2 = box1
    x3, y3, x4, y4 = box2
    x_overlap = max(0, min(x2, x4) - max(x1, x3))
    y_overlap = max(0, min(y2, y4) - max(y1, y3))
    area1 = (x2 - x1) * (y2 - y1)
    area2 = (x4 - x3) * (y4 - y3)
    overlap = x_overlap * y_overlap
    iou = overlap / (area1 + area2 - overlap)
    return iou

In [24]:
print(f'Number of crops: {len(image_crop)}')

from ultralytics import YOLO

# Load a model
model = YOLO('/home/skyarrow/Workspace/Yolov8-Drillbit-Detection/Predict/Weights/train2_fulldata_batch4_epoch200.pt')  # load the best checkpoint
model.max_boxes = 500
model.max_detection_points = 500
predst = f'{HOME}/Images/Image_Crop_Predicted/'
%cd {HOME}/Images/Image_Crop_Predicted/


image_big = image_original[0]
image_draw = image_original[0].copy()
bboxes = []
idx = 0
touch_edge = 10
touch_edge1 = 30
hw_ratio = 1.4
IOU_ratio = 0.2
total_area = 0
cover_pixel = 3
color = {'0': (162, 84, 66), '1': (25, 90, 192), '2': (168, 73, 138), '3': (255, 255, 0), '4': (0, 255, 255)}
cls_count = {'0': 0, '1': 0, '2': 0}
total_count = 0
avg_area = 0

def Touch_Edge(box):
    if (abs(int(box[0]) - 0) <= touch_edge) or (abs(int(box[1]) - 0) <= touch_edge) or (abs(int(box[2]) - 640) <= touch_edge) or (abs(int(box[3]) - 640) <= touch_edge):
        # print(f"Touch edge: {box}" )
        return True
    else:
        return False
    
def Shape_Unlikely(box):
    global total_area, total_count, avg_area
    h = box[3] - box[1]
    w = box[2] - box[0]
    area = h * w
    if (h/w > hw_ratio) or (w/h > hw_ratio):
        # print(f"Shape unlikely: {box}")
        return True
    else:
        if total_count > 50:
            avg_area = total_area / total_count
            if area > avg_area * 1.4:
                return True
        else:
            return False  




for img in image_crop:
    # now_img = img.copy()
    now_img = img
    results = model(now_img)  # return a list of Results objects

    # Process results list
    for result in results:
        # print(len(results))
        boxes = result.boxes  # Boxes object for bounding box outputs
        classes = boxes.cls
        
        now_idx = 0
        for box in boxes.xyxy:
            x1 = int(box[0]) + image_crop_topleft[idx][1]
            y1 = int(box[1]) + image_crop_topleft[idx][0]
            x2 = int(box[2]) + image_crop_topleft[idx][1]
            y2 = int(box[3]) + image_crop_topleft[idx][0]
            now_box = (x1, y1, x2, y2)
            now_cls = boxes.cls[now_idx]
            now_cls = int(now_cls)
            
            
            # if now_cls == 1:
            #     print('111')
            #     cv2.rectangle(image_draw, (x1, y1), (x2, y2), color[str(now_cls)], 3)
            
            if(Touch_Edge(box) or Shape_Unlikely(box)):
                continue
            
            overlap = False
            for bbox in bboxes:
                if Calculate_IOU(bbox, now_box) > IOU_ratio:
                    overlap = True
                    # print(f"Overlap: {bbox} and {now_box}")
                    break
                
            
            edge_cnt = 0
            if not overlap:
                bboxes.append(now_box)
                total_area += (x2 - x1) * (y2 - y1)
                
                
                # print(f'Class: {now_cls}')
                cls_count[str(now_cls)] += 1
                total_count += 1
                
                cv2.rectangle(image_draw, (x1, y1), (x2, y2), color[str(now_cls)], 3)
                # now_img = cv2.rectangle(now_img, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)
                image_big[y1+cover_pixel:y2-cover_pixel, x1+cover_pixel:x2-cover_pixel, :] = (0, 0, 0)
                
                
                
                
            now_idx += 1
    cv2.imwrite(os.path.join(predst, f'image{idx}_predicted.jpg'), now_img)
    idx += 1



print(f"Total Number = {total_count}")
print(f"Boxes Number = {len(bboxes)}")
print(f"Average Area = {total_area / total_count}")
for cls, num in cls_count.items():
    print(f'Class {name_dict[str(cls)]} = {num}')

# print(f'Answer: tool:1 tool_b:498 void:1')
# answer_num = 500
# print(f'Answer: tool:5929 tool_b:11 void:13')
# answer_num = 5953
# print(f'Answer: tool:5878 tool_b:21 void:16')
# answer_num = 5915


%cd {HOME}/Images/Image_Results/
cv2.putText(image_draw, f"Total Number = {total_count}", (150, 150), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 2, cv2.LINE_AA)
cv2.putText(image_draw, f"Tool Number = {cls_count['0']}", (150, 250), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2, cv2.LINE_AA)
cv2.putText(image_draw, f"Tool_b Number = {cls_count['1']}", (150, 350), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2, cv2.LINE_AA)
cv2.putText(image_draw, f"Void Number = {cls_count['2']}", (150, 450), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2, cv2.LINE_AA)
# cv2.putText(image_draw, f'Answer: tool:1 tool_b:498 void:1', (150, 550), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2, cv2.LINE_AA)
# cv2.putText(image_draw, 'Number Precision : {:.3%}'.format(1.0 - (abs(total_count - answer_num)/answer_num)), (150, 650), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2, cv2.LINE_AA)
cv2.imwrite('result.jpg', image_draw)
cv2.imwrite('filtered.jpg', image_big)

%cd {HOME}
    


0: 640x640 (no detections), 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 tool, 6.3ms
Speed: 1.2ms preprocess, 6.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 33 tools, 6.3ms
Speed: 1.0ms preprocess, 6.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)



Number of crops: 160
/home/skyarrow/Workspace/Yolov8-Drillbit-Detection/Predict/Images/Image_Crop_Predicted


0: 640x640 60 tools, 6.7ms
Speed: 1.1ms preprocess, 6.7ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 65 tools, 7.1ms
Speed: 1.1ms preprocess, 7.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 60 tools, 2 tool_bs, 7.3ms
Speed: 1.0ms preprocess, 7.3ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 62 tools, 1 tool_b, 6.4ms
Speed: 1.1ms preprocess, 6.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 60 tools, 2 tool_bs, 7.1ms
Speed: 1.0ms preprocess, 7.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 63 tools, 6.7ms
Speed: 1.1ms preprocess, 6.7ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 60 tools, 6.6ms
Speed: 1.7ms preprocess, 6.6ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 66 tools, 6.4ms
Speed: 1.0ms preprocess, 6.4ms inference, 0.7ms postprocess per image at 

Total Number = 1425
Boxes Number = 1425
Average Area = 4667.825263157894
Class tool = 1425
Class tool_b = 0
Class void = 0
/home/skyarrow/Workspace/Yolov8-Drillbit-Detection/Predict/Images/Image_Results
/home/skyarrow/Workspace/Yolov8-Drillbit-Detection/Predict
